In [ ]:
# import tensorflow module
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

# import tensorflow reinforcement learning modules
from tf_agents.agents.dqn import dqn_agent
from tf_agents.agents.ppo import ppo_agent, ppo_actor_network
from tf_agents.networks import value_network, actor_distribution_network
from tf_agents.environments import suite_gym
from tf_agents.environments import tf_py_environment
from tf_agents.eval import metric_utils
from tf_agents.metrics import tf_metrics
from tf_agents.networks import sequential
from tf_agents.policies import random_tf_policy
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.trajectories import trajectory
from tf_agents.specs import tensor_spec
from tf_agents.utils import common
from tf_agents.replay_buffers.tf_uniform_replay_buffer import TFUniformReplayBuffer
from tf_agents.policies import PolicySaver
# import gym

# import useful module
import os
import numpy as np
import matplotlib.pyplot as plt
import PIL.Image as plImage
from IPython.display import display

測試import有無成功

In [ ]:
# test if the module being imported successfully # 可以註解掉

# env_name = 'CartPole-v0'
# env = suite_gym.load(env_name)
# # env.reset()                     # 回到初始state
# # plImage.fromarray(env.render()) # 可以秀出當前state的圖片
# # env.close()                     # 要關閉才不會當掉
# #由環境本身紀錄的各種規格 => 用於創建神經網路(知道輸入跟輸出的各種資訊)

# print('All Spec at this time step: \n', env.time_step_spec(), '\n')   # 這個時間點得到的各種東西的規格)
# print('Observation Spec: \n', env.time_step_spec().observation, '\n') # 此 Env 輸出的 state 會是這個規格
# print('Reward Spec: \n', env.time_step_spec().reward, '\n')           # 此 Env 上一個action所造成的 reward 的規格
# print('Action Spec: \n', env.action_spec(), '\n')                     # 要輸入此 Env 的 action 要符合的規格

# # 由環境真實輸出的state (會符合上方的規格))
# time_step = env.reset()
# print('Time step:\n', time_step, '\n')

# # take an action 
# action = np.array(1, dtype=np.int32)
# next_time_step = env.step(action)
# print('Next time step:\n', next_time_step)

In [ ]:
from connextx_game_board import GameBoardEnv
from connectx_agents import agent_random, MinimaxAgent
from tf_agents.environments import TFPyEnvironment
from tf_agents.environments.gym_wrapper import GymWrapper

def CreateTFGameBoardEnv(opponent_agent):
        gym_env        = GameBoardEnv(opponent_agent)   # 一開始是Gym架構的Env
        suit_gym_env   = GymWrapper(gym_env)            # 轉成tensorflow包裝好的Suit gym env架構
        tf_tensor_env  = TFPyEnvironment(suit_gym_env)  # 再轉成tf tensor env
        return tf_tensor_env

def create_my_train_evaluate_envs(opponent_agent):
    train_env = CreateTFGameBoardEnv(opponent_agent)
    evaluate_env = CreateTFGameBoardEnv(opponent_agent)
    return train_env, evaluate_env

# train_env, evaluate_env = create_my_train_evaluate_envs(agent_random)

In [ ]:
# 由於要用 DQN演算法 要建立 Qnet 以及建立 DQN Agent的部分
# action_tensor_spec = tensor_spec.from_spec(env.action_spec())               # 從測試用的env取得spec 再轉換成tensorflow架構
# action_tensor_spec = train_env.action_spec()                                # 也可以直接從剛剛轉換成tf_Env的環境取得tensorflow架構的spec
# num_actions = action_tensor_spec.maximum - action_tensor_spec.minimum + 1   # 根據spec算出總共有多少可使用的action
from tensorflow.keras.layers import Conv2D, Dense, Flatten, BatchNormalization, Activation
from tensorflow.keras.activations import softmax, tanh

def dense_layer(num_output_units): # 設定輸出的數值數量
    return Dense(
        num_output_units,
        activation=tf.keras.activations.relu,
        kernel_initializer=tf.keras.initializers.VarianceScaling(scale=2.0, mode='fan_in', distribution='truncated_normal')
    )
# QNetwork consists of a sequence of Dense layers followed by a dense layer
# with `num_actions` units to generate one q_value per available action as it's output.
def create_my_qnetwork(num_output):
    # Define a helper function to create Dense layers configured with the right activation and kernel initializer.
    
    layer_list = [
        # Conv2D(16, (3, 3), padding='same', strides=(2, 2), activation='relu'),
        # Conv2D(8, (3, 3), padding='same', strides=(2, 2), activation='relu'),
        Flatten(),
        dense_layer(128),
        dense_layer(64),
        Dense(
            num_output,                     # 最後有幾個action就輸出幾個數值
            activation=None,                # q net 最後輸出 意義是每個動作預期的reward
            kernel_initializer=tf.keras.initializers.RandomUniform(minval=-0.03, maxval=0.03),
            bias_initializer=tf.keras.initializers.Constant(-0.2),
        )
    ]

    q_net = sequential.Sequential(layer_list)
    return q_net

def create_initialized_dqn_agent(time_step_spec, action_spec, q_net, learning_rate):
    agent = dqn_agent.DqnAgent(
        time_step_spec      = time_step_spec,
        action_spec         = action_spec,
        q_network           = q_net,
        optimizer           = tf.keras.optimizers.Adam(learning_rate=learning_rate),
        td_errors_loss_fn   = common.element_wise_squared_loss,
        train_step_counter  = tf.Variable(0)  # 看總共訓練幾次的計數器
    )
    #將所有的參數初始化
    agent.initialize()

    # initialize before training
    agent.train = common.function(agent.train)     # 將訓練的function 加到tensor board graph中(可有可無)
    agent.train_step_counter.assign(0)             # Reset the train step，初始化紀錄訓練次數的counter
    return agent


def create_my_actor_critic_net(train_env: TFPyEnvironment):
    # actor_layer_list = [
    #     Flatten(),
    #     dense_layer(200),
    #     dense_layer(100),
    #     dense_layer(50),
    #     dense_layer(num_output),
    #     Activation(softmax)
    # ]    
    # actor_net = sequential.Sequential(actor_layer_list)

    # critic_layer_list = [
    #     Flatten(),
    #     dense_layer(100),
    #     dense_layer(50),
    #     dense_layer(10),
    #     dense_layer(1),
    #     Activation(tanh)
    # ]
    # critic_net = sequential.Sequential(critic_layer_list)

    # value_network, ppo_actor_network
    actor_net = actor_distribution_network.ActorDistributionNetwork(
        train_env.time_step_spec().observation,
        train_env.action_spec()
    )

    critic_net = value_network.ValueNetwork(
        train_env.time_step_spec().observation
    )
    return actor_net, critic_net

def create_initialized_ppo_agent(time_step_spec, action_spec, actor_net, critic_net, learning_rate):
    agent = ppo_agent.PPOAgent(
        time_step_spec     = time_step_spec,
        action_spec        = action_spec,
        value_net          = critic_net,
        actor_net          = actor_net,
        log_prob_clipping  = 1,
        use_gae            = True,
        train_step_counter = tf.Variable(0)
    )

    #將所有的參數初始化
    agent.initialize()

    # initialize before training
    agent.train = common.function(agent.train)     # 將訓練的function 加到tensor board graph中(可有可無)
    agent.train_step_counter.assign(0)             # Reset the train step，初始化紀錄訓練次數的counter

    return agent


In [ ]:
# 蒐集資料的部分
def take_one_step(environment, policy):                     # 把當前的Env 根據某個policy採取一個動作
    time_step = environment.current_time_step()             # get current state
    action_step = policy.action(time_step)                  # get action of this state from the policy
    next_time_step = environment.step(action_step.action)   # take action，環境的狀態真的會改變(並不只是看看下個狀態)
    
    return time_step, action_step, next_time_step           # 回傳current_state, action, next_state

def collect_data(train_env, collect_policy, replay_buffer, num_steps):
    for _ in range(num_steps):
        state, action, next_state = take_one_step(train_env, collect_policy)

        traj = trajectory.from_transition(state, action, next_state) # 把state action next_state存成一筆trajectory data
        replay_buffer.add_batch(traj)                                # Add trajectory to the replay buffer

def create_replay_buffer_and_training_iterater(traj_spec, train_env, buffer_length, initial_random_steps):
    replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer( # 要設定每一筆資料的格式
        data_spec   = traj_spec,             # 每個演算法要的格式會不一樣，所以要跟每個演算法拿藥蒐集的資料格式，collect_data_spec是一個named tuple，名字是Trajectory
        batch_size  = train_env.batch_size, # 一次要拿多少筆資料進去訓練
        max_length  = buffer_length
    )        # 最多有多少筆資料在Buffer裡面

    #可以自己創造一個隨機選取動作的policy(由訓練環境的輸入和輸出規格設定)
    random_policy = random_tf_policy.RandomTFPolicy(train_env.time_step_spec(), train_env.action_spec()) 
    # random_policy.action(train_env.current_time_step()) # TF Policy吃的就是 TF time step決定他的action

    # 在訓練前先蒐集一定數量的隨機動作
    collect_data(train_env, random_policy, replay_buffer, initial_random_steps)

    # 這個Buffer 要轉成tensorflow dataset的形式作為訓練model的data pipeline
    training_dataset = replay_buffer.as_dataset(
        num_parallel_calls  =3, 
        sample_batch_size   =train_env.batch_size, 
        num_steps           =2
    ).prefetch(3)                # num step=2 是因為DQN一次需要這個state跟下個state計算loss

    training_iterater = iter(training_dataset)  # 訓練的pipeline
    return replay_buffer, training_iterater


In [ ]:
# 定義要evaluate這個policy好不好的function，玩完一定的次數後算平均的reward

def compute_average_return(environment, policy, num_episodes=10):
    total_return = 0.0  

    for _ in range(num_episodes):

        time_step = environment.reset()
        episode_return = 0.0

        while not time_step.is_last():
            action_step = policy.action(time_step)
            time_step = environment.step(action_step.action)
            episode_return += time_step.reward
        total_return += episode_return


    average_return = total_return / num_episodes
    return average_return.numpy()[0]
# average_reward = compute_average_return(evaluate_env, random_policy, num_evaluate_episodes) # 測試是否成功

def train_one_iteration(train_env, agent, replay_buffer, train_date_iterater, num_collect_steps):
    collect_data(train_env, agent.collect_policy, replay_buffer, num_collect_steps) # 每一個訓練迴圈會利用collect data policy取得一些訓練資料
    
    # Sample a batch of data from the buffer and update the agent's network.
    experience, __unused_info = next(train_date_iterater) # 根據訓練資料pipeline取得一筆訓練資料
    training_loss = agent.train(experience).loss        # 用訓練資料做訓練並且算出訓練的loss
    return training_loss

def save_policy(policy_name, policy):
    if not os.path.isdir(policy_name):
        policy_saver = PolicySaver(policy, batch_size=None)
        policy_saver.save(policy_name)
    else:
        print('Policy Exist Already')

In [ ]:
# setting hyperparameters
num_training_iterations = 50000      # @param {type:"integer"}   # 總共要訓練幾個迴圈
batch_size = 64                      # @param {type:"integer"}   # 
learning_rate = 1e-3                 # @param {type:"number"}    # learning rate for optimizer of the net
log_interval = 200                   # @param {type:"integer"}   # show loss after every x training steps 

# collect data parameters
initial_collect_steps = 100          # @param {type:"integer"}   # warm up random steps to test the enviromments
collect_steps_per_iteration = 1      # @param {type:"integer"}   # collect x step data every trainging loop 
replay_buffer_max_length = 10000     # @param {type:"integer"}   # 

# evaluate parameters
num_evaluate_episodes = 10           # @param {type:"integer"}   # test current average reward of the current policy by playing x times
evaluate_interval = 1000             # @param {type:"integer"}   # show average reward every x training steps

In [ ]:
# train initialize process
train_env, evaluate_env = create_my_train_evaluate_envs(opponent_agent=MinimaxAgent(1))

action_tensor_spec = train_env.action_spec()                                # 也可以直接從剛剛轉換成tf_Env的環境取得tensorflow架構的spec
num_actions = action_tensor_spec.maximum - action_tensor_spec.minimum + 1   # 根據spec算出總共有多少可使用的action

# # dqn method
q_net = create_my_qnetwork(num_actions)

agent = create_initialized_dqn_agent( \
      train_env.time_step_spec(),
      train_env.action_spec(),
      q_net,
      learning_rate
   )
# actor_net, critic_net = create_my_actor_critic_net(train_env)
# agent = \
#    create_initialized_ppo_agent(
#       train_env.time_step_spec(),
#       train_env.action_spec(),
#       actor_net,
#       critic_net,
#       learning_rate
#    )


replay_buffer, training_iterater = \
   create_replay_buffer_and_training_iterater( \
      agent.collect_data_spec,
      train_env,
      replay_buffer_max_length,
      initial_collect_steps
   )

In [ ]:
# average_return = compute_average_return(evaluate_env, agent.policy, num_evaluate_episodes)
# returns = [average_return] # 儲存訓練過程中的平均分數成長
# 定義一個整個訓練過程 (train_env, agent, replay_buffer, training_iterater, collect_steps_per_iteration, log_interval, evaluate_interval 要先定義好)
def train_agent(iterations):
    average_return = compute_average_return(evaluate_env, agent.policy, num_evaluate_episodes)
    returns = [average_return] # 儲存訓練過程中的平均分數成長

    for _ in range(iterations):

        training_loss = train_one_iteration(train_env, agent, replay_buffer, training_iterater, collect_steps_per_iteration)

        #看看訓練到哪步了
        step = agent.train_step_counter.numpy()
        if step % log_interval == 0:
            print('step = {0}: loss = {1}'.format(step, training_loss))

        if step % evaluate_interval == 0:
            average_return = compute_average_return(evaluate_env, agent.policy, num_evaluate_episodes)
            print('step = {0}: Average Return = {1}'.format(step, average_return))
            returns.append(average_return)
    return returns

In [ ]:
# train_env, evaluate_env = create_my_train_evaluate_envs(opponent_agent=MinimaxAgent(1))

In [ ]:
# 訓練Agent

#%% time # 用來輸出總訓練時間
train_iterations = num_training_iterations*2

returns = train_agent(train_iterations)

#訓練完畢後，將訓練過程中計算的rewards 畫成折線圖顯示出來
xrange_value = range(0, train_iterations+1, evaluate_interval)
plt.ylabel('Average Return'); plt.xlabel('Iterations'); plt.plot(xrange_value, returns)

# 訓練完畢就可以把訓練好的policy儲存起來
save_policy(policy_name=f'DqnPolicy_{agent.train_step_counter.numpy()}', policy=agent.policy)

In [ ]:
# 純粹試跑這個訓練好的Policy，與訓練無關，所以重新import

import os
from tf_agents.environments import suite_gym
from tf_agents.environments import tf_py_environment
import PIL.Image as plImage
import tensorflow as tf
# 
try:
    policy_name = 'DqnPolicy_%d' % 20000
    env_name = 'CartPole-v0'
    env = suite_gym.load(env_name)
    evaluate_env = tf_py_environment.TFPyEnvironment(suite_gym.load(env_name))
    saved_policy = tf.compat.v2.saved_model.load(policy_name)
    # print(compute_average_return(evaluate_env, saved_policy, 10))

    for ep in range(10):
        env.reset()
        time_step = evaluate_env.reset()
        
        while not time_step.is_last():

            # env.render()
            # plImage.fromarray(evaluate_env.render().numpy())
            plImage.fromarray(env.render())
            
            action_step = saved_policy.action(time_step)
            time_step = evaluate_env.step(action_step)
            env.step(action_step.action.numpy()[0])
except Exception as e:
    print(e)
finally:
    env.close()


In [ ]:
# import gym

# env = gym.make('CartPole-v0')
# total_episode = 100

# for episode in range(total_episode):
#     env.reset()
    
#     for timestep in range(1, 1000+1):
#         env.render()
#         observation, reward, done, info = env.step(env.action_space.sample())
#         if done:
#                 print("Episode finished after {} timesteps".format(timestep))
#                 break
#     break
# env.close()